In [1]:
#Dependencies
import pandas as pd
import numpy as np
import os

In [2]:
#Source data files
school_data = os.path.join('.\Resources','schools_complete.csv')
student_data = os.path.join('.\Resources','students_complete.csv')


In [3]:
#Read in csv data
load_school = pd.read_csv(school_data)
load_student = pd.read_csv(student_data)

In [4]:
load_school.head()

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500


In [5]:
#Total number of schools
num_schools = load_school['school_name'].count()
#num_schools = load_school.count(axis='columns')
print(num_schools)

15


In [6]:
#Total District Budget
total_budget = load_school['budget'].sum()
#print(total_budget)

In [7]:
load_student.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84


In [8]:
#Number of students
num_students = load_student['student_name'].count()
print(num_students)

39170


In [9]:
#Average math score
#avg_math = load_student['math_score'].mean()
#print(avg_math)
#Average reading score
#avg_read = load_student['reading_score'].mean()
#print(avg_read)
avg_both = load_student.mean()
#print(avg_both)
print(avg_both.iloc[1:3])

reading_score    81.877840
math_score       78.985371
dtype: float64


In [10]:
#Percent passing reading
prcnt_pass_read = load_student.loc[load_student['reading_score'] >= 70,'student_name'].count()/num_students
#format it later
print(f"{round(prcnt_pass_read,3)*100}%")
#rename column
#percent_pass = prcnt_pass_read.rename(columns={'student_name':'percent_pass'})
#print(percent_pass)

85.8%


In [11]:
#Percent passing reading
prcnt_pass_math = load_student.loc[load_student['math_score'] >= 70,'student_name'].count()/num_students

print(f"{round(prcnt_pass_math,3)*100}%")

75.0%


In [12]:
#Overall pass rate
overall = round(((prcnt_pass_read + prcnt_pass_math)/2)*100,1)
print(f"{overall}%")

80.4%


In [13]:
#No duplicate schools
no_dups_df = load_school.drop_duplicates(subset='school_name',keep='first')
no_dups_df

,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [23]:
#Add budget per student
no_dups_df['per_stu'] = no_dups_df['budget']/no_dups_df['size']
#set new index as school name
no_dups_df = no_dups_df.set_index('school_name')


In [35]:
#Merge two data sets based on school_name column
all_data = load_student.merge(load_school,on='school_name',how='inner')
all_data.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [26]:
#Generate df of mean scores
mean_scores = all_data.groupby('school_name').mean()
mean_scores.head()

,Student ID,reading_score,math_score,School ID,size,budget
school_name,,,,,,
Bailey High School,20358.5,81.033963,77.048432,7.0,4976.0,3124928.0
Cabrera High School,16941.5,83.975780,83.061895,6.0,1858.0,1081356.0
Figueroa High School,4391.0,81.158020,76.711767,1.0,2949.0,1884411.0
Ford High School,36165.0,80.746258,77.102592,13.0,2739.0,1763916.0
Griffin High School,12995.5,83.816757,83.351499,4.0,1468.0,917500.0


In [27]:
#Select columns and create new df
sub_mean_df = mean_scores[['reading_score','math_score']]
#Add mean columns to schools list
final_df = pd.concat([no_dups_df,sub_mean_df],axis=1,sort=True)
final_df

,School ID,type,size,budget,per_stu,reading_score,math_score
Bailey High School,7,District,4976,3124928,628.0,81.033963,77.048432
Cabrera High School,6,Charter,1858,1081356,582.0,83.975780,83.061895
Figueroa High School,1,District,2949,1884411,639.0,81.158020,76.711767
Ford High School,13,District,2739,1763916,644.0,80.746258,77.102592
Griffin High School,4,Charter,1468,917500,625.0,83.816757,83.351499
Hernandez High School,3,District,4635,3022020,652.0,80.934412,77.289752
Holden High School,8,Charter,427,248087,581.0,83.814988,83.803279
Huang High School,0,District,2917,1910635,655.0,81.182722,76.629414
Johnson High School,12,District,4761,3094650,650.0,80.966394,77.072464
Pena High School,9,Charter,962,585858,609.0,84.044699,83.839917


In [65]:
#Get percent passing reading and math

pass_df = all_data.groupby('school_name')[['reading_score','math_score']].apply(lambda x: (x >= 70).sum()).reset_index()
#df11=df.groupby('key1')['key2'].apply(lambda x: (x=='one').sum()).reset_index(name='count')
pass_df = pass_df.set_index(['school_name'])
pass_df['math_%_pass'] = round((pass_df['math_score']/final_df['size'])*100,1)
pass_df['read_%_pass'] = round((pass_df['reading_score']/final_df['size'])*100,1)   
pass_df

,reading_score,math_score,math_%_pass,read_%_pass
school_name,,,,
Bailey High School,4077,3318,66.7,81.9
Cabrera High School,1803,1749,94.1,97.0
Figueroa High School,2381,1946,66.0,80.7
Ford High School,2172,1871,68.3,79.3
Griffin High School,1426,1371,93.4,97.1
Hernandez High School,3748,3094,66.8,80.9
Holden High School,411,395,92.5,96.3
Huang High School,2372,1916,65.7,81.3
Johnson High School,3867,3145,66.1,81.2


In [66]:
#Select columns and create new df
sub_prcnt_df = pass_df[['math_%_pass','read_%_pass']]
#Add mean columns to schools list
final_school_df = pd.concat([final_df,sub_prcnt_df],axis=1,sort=True)
final_school_df

,School ID,type,size,budget,per_stu,reading_score,math_score,math_%_pass,read_%_pass
school_name,,,,,,,,,
Bailey High School,7,District,4976,3124928,628.0,81.033963,77.048432,66.7,81.9
Cabrera High School,6,Charter,1858,1081356,582.0,83.975780,83.061895,94.1,97.0
Figueroa High School,1,District,2949,1884411,639.0,81.158020,76.711767,66.0,80.7
Ford High School,13,District,2739,1763916,644.0,80.746258,77.102592,68.3,79.3
Griffin High School,4,Charter,1468,917500,625.0,83.816757,83.351499,93.4,97.1
Hernandez High School,3,District,4635,3022020,652.0,80.934412,77.289752,66.8,80.9
Holden High School,8,Charter,427,248087,581.0,83.814988,83.803279,92.5,96.3
Huang High School,0,District,2917,1910635,655.0,81.182722,76.629414,65.7,81.3
Johnson High School,12,District,4761,3094650,650.0,80.966394,77.072464,66.1,81.2


In [68]:
#Add overall passing based on % math and % read passing
final_school_df['overall_pass'] = (final_school_df['math_%_pass'] + final_school_df['read_%_pass'])/2
final_school_df

,School ID,type,size,budget,per_stu,reading_score,math_score,math_%_pass,read_%_pass,overall_pass
school_name,,,,,,,,,,
Bailey High School,7,District,4976,3124928,628.0,81.033963,77.048432,66.7,81.9,74.30
Cabrera High School,6,Charter,1858,1081356,582.0,83.975780,83.061895,94.1,97.0,95.55
Figueroa High School,1,District,2949,1884411,639.0,81.158020,76.711767,66.0,80.7,73.35
Ford High School,13,District,2739,1763916,644.0,80.746258,77.102592,68.3,79.3,73.80
Griffin High School,4,Charter,1468,917500,625.0,83.816757,83.351499,93.4,97.1,95.25
Hernandez High School,3,District,4635,3022020,652.0,80.934412,77.289752,66.8,80.9,73.85
Holden High School,8,Charter,427,248087,581.0,83.814988,83.803279,92.5,96.3,94.40
Huang High School,0,District,2917,1910635,655.0,81.182722,76.629414,65.7,81.3,73.50
Johnson High School,12,District,4761,3094650,650.0,80.966394,77.072464,66.1,81.2,73.65
